In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
import matplotlib.pyplot as plt

In [46]:
def dataclean(zillow_yvalues_data,census_data,date,currentrentdate):
    # takes in the zillow data with all the years and acs census data of two years before 
    # year to predict and date formatted '2020-09' and joins the census on zillow and cleans 
    
    
    zillow_yvalues_data_yearsubsect=zillow_yvalues_data[['RegionName','City','State',date,currentrentdate]]
    fulldata=zillow_yvalues_data_yearsubsect.merge(census_data,how='left',left_on='RegionName', right_on='geo_id' ).drop(['geo_id'],axis=1)
    fulldata=fulldata[fulldata[date].isna()!=1]
    #drop column if >20% are nas 
    cols_to_drop=['do_date']
    for i in fulldata.columns:

        if sum(fulldata[i].isna())/len(fulldata)> .2:
            cols_to_drop.append(i)
    print(cols_to_drop)
    fulldata=fulldata.drop(cols_to_drop,axis=1)
    #drop rows with more than 50% nas
    #in this jan2020was one row
    fulldata=fulldata.loc[fulldata.isna().sum(axis=1)<.5*len(fulldata.columns)]
    #num of missing values in columns
    countofcols=0
    cols_with_na=[]
    for i in fulldata.columns:
        if i in cols_to_drop:
            print(i)
        elif sum(fulldata[i].isna())>0:
#             print('Column:',i)
#             print('Num of NAs:',sum(fulldata[i].isna()))
#             print('Num of 0s:',sum(fulldata[i]==0))
            cols_with_na.append(i)
            countofcols+=1
    #fill in nas with avrg of the column for the city 
    for i in cols_with_na:
        for indexofna in (list(fulldata[i][fulldata[i].isna()].index)):

            if bool(~(fulldata[fulldata.City==fulldata.loc[indexofna,'City']][i].mean()==fulldata[fulldata.City==fulldata.loc[indexofna,'City']][i].mean())):
                fulldata.loc[indexofna,i]=fulldata[fulldata.State==fulldata.loc[indexofna,'State']][i].mean()
                
            else:
                fulldata.loc[indexofna,i]=fulldata[fulldata.City==fulldata.loc[indexofna,'City']][i].mean()
                
    
    return fulldata

In [54]:


fulldata=dataclean(pd.read_csv('Zip_Zri_MultiFamilyResidenceRental-2.csv',index_col=False),pd.read_csv('acs_data/acs2015.csv',index_col=False),'2017-09','2015-09')

fulldata

['do_date', 'geoid']


,RegionName,City,State,2017-09,2015-09,nonfamily_households,family_households,median_year_structure_built,rent_burden_not_computed,rent_over_50_percent,...,pop_25_years_over,commute_35_44_mins,commute_60_more_mins,commute_less_10_mins,commuters_16_over,hispanic_any_race,pop_5_years_over,speak_only_english_at_home,speak_spanish_at_home,speak_spanish_at_home_low_english
0,10025,New York,NY,3787.0,3694.000000,24367.0,19089.0,1939.0,1754.0,7527.0,...,71927.0,6084.0,4294.0,2468.0,45657.0,21349.0,91087.0,59561.0,16641.0,6676.0
1,60657,Chicago,IL,1804.0,1755.000000,25743.0,11011.0,1954.0,837.0,4002.0,...,52822.0,6809.0,5175.0,1728.0,45212.0,4193.0,65248.0,55901.0,2905.0,785.0
2,10023,New York,NY,3651.0,3652.000000,20407.0,12348.0,1944.0,919.0,3221.0,...,48957.0,2468.0,1858.0,896.0,30744.0,5880.0,56493.0,41807.0,4269.0,926.0
3,77494,Katy,TX,1182.0,1304.000000,3806.0,23386.0,2006.0,57.0,672.0,...,52360.0,4029.0,6625.0,2256.0,36706.0,15264.0,80564.0,52511.0,11597.0,2367.0
4,60614,Chicago,IL,1880.0,1828.000000,22453.0,11252.0,1962.0,696.0,3276.0,...,48786.0,4162.0,4247.0,1839.0,42223.0,4634.0,65400.0,56832.0,2580.0,783.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1853,78215,San Antonio,TX,1308.0,1160.991228,681.0,334.0,1980.0,62.0,198.0,...,1531.0,39.0,43.0,155.0,1220.0,1033.0,1772.0,1250.0,453.0,68.0
1854,2142,Cambridge,MA,3284.0,3133.000000,841.0,466.0,2000.0,50.0,336.0,...,1952.0,110.0,43.0,264.0,1241.0,297.0,2794.0,1460.0,270.0,14.0
1855,18232,Lansford,PA,798.0,777.000000,737.0,898.0,1939.0,20.0,139.0,...,2762.0,188.0,87.0,231.0,1421.0,276.0,3656.0,3521.0,78.0,0.0
1856,75202,Dallas,TX,1493.0,1504.000000,1125.0,278.0,1968.0,0.0,201.0,...,1825.0,47.0,64.0,316.0,1597.0,314.0,2121.0,1732.0,186.0,20.0


# Assembling training data w yvals from 2011-2017 and before 
-means census data from 2015 and before 


In [58]:
def assemble_train_test_predict(years_out=3,year_gap_in_data=2):
    
    df = pd.read_csv('Zip_Zri_MultiFamilyResidenceRental-2.csv',index_col=False)
    zillow_yvalues_data=pd.merge(df.iloc[:,0:7], df.iloc[:,8:].interpolate(axis=1), left_index=True, right_index=True)
    genericfilepath='acs_data/acs20??.csv'
    census_data_years=list(range(12,19,1))
    years=list(range(14,20,1))
    years_for_train=years[:-years_out]
    years_for_pred=years[-years_out:]
    census_data_years_train=census_data_years[:-years_out]
    print(years_for_train)
    print(census_data_years_train)
    olddatedata=pd.DataFrame()
    for i in years_for_train:
        print(i)
        if str(i-year_gap_in_data+1-years_out)!='11':
            tempfilepath=genericfilepath.replace('??', str(i-year_gap_in_data+1-years_out))
            print('census file path',tempfilepath)
            yearnumber=str(i)
            if len(yearnumber)<2:
                yearnumber='0'+yearnumber
            date='20'+yearnumber+'-09'
            currentrentdate='20'+str(i-year_gap_in_data+1-years_out)+'-09'
            print('zillow date',date)
            print('current rent date', currentrentdate)
    #         print('date',date)
    #         print('tempfilepath',tempfilepath)
            try:
                census_data= pd.read_csv(tempfilepath,index_col=False)

                datedata=dataclean(zillow_yvalues_data,census_data,date,currentrentdate)
                datedata['RegionName']=datedata['RegionName'].astype(str)+'-'+date
                datedata=datedata.rename(columns={date: 'Rent',currentrentdate:'Current Rent'})
                print(datedata.columns)
                olddatedata=pd.concat([datedata,olddatedata], ignore_index=True, sort=False)
            except:
                placeholder=1
    
    
    fulldata=olddatedata
    cols_to_drop=[]
    for i in fulldata.columns:
        if sum(fulldata[i].isna())/len(fulldata)> .2:
            cols_to_drop.append(i)
    fulldata=fulldata.drop(cols_to_drop,axis=1)
    
    
#     print('\n\npred\n\n')
    
    #assemble prediction dataset
    olddatedata=pd.DataFrame()
    for i in years_for_pred:
        print(str(i-year_gap_in_data+1-years_out))
        if str(i-year_gap_in_data+1-years_out)!='11':
            print('noteleven')
            tempfilepath=genericfilepath.replace('??', str(i-year_gap_in_data+1-years_out))
            yearnumber=str(i)
            if len(yearnumber)<2:
                yearnumber='0'+yearnumber
            date='20'+yearnumber+'-09'
            currentrentdate='20'+str(i-year_gap_in_data+1-years_out)+'-09'
            print('date',date)
            if date=='2020-09':
                date='2020-01'
    #         print('date',date)
    #         print('tempfilepath',tempfilepath)

            census_data= pd.read_csv(tempfilepath,index_col=False)
            datedata=dataclean(zillow_yvalues_data,census_data,date,currentrentdate)
            datedata['RegionName']=datedata['RegionName'].astype(str)+'-'+date
            datedata=datedata.rename(columns={date: 'Rent',currentrentdate:'Current Rent'})
            print(datedata.columns)
    #         newfilename='Data_acsPREDICT20'+yearnumber+'.csv'
    #         datedata.to_csv(newfilename)
            olddatedata=pd.concat([datedata,olddatedata], ignore_index=True, sort=False)

    
    
    preddata=olddatedata
    cols_to_drop=[]
    for i in preddata.columns:
        if sum(preddata[i].isna())/len(preddata)> .2:
            cols_to_drop.append(i)
    preddata=preddata.drop(cols_to_drop,axis=1)
    
    
    #ensures prediciton and training set have same cols 
    cols_in_common=list(fulldata.columns & preddata.columns)
    print(len(preddata.columns))
    print(len(cols_in_common))
    preddata=preddata[cols_in_common]
    fulldata=fulldata[cols_in_common]

    
    
#     #train test split
    tdata_train, tdata_test = train_test_split(fulldata, test_size=0.33, random_state=100)
    tdata_train.to_csv('Data_acsTRAIN7.17_3year.csv',index=False)
    tdata_test.to_csv('Data_acsTEST7.17_3year.csv',index=False)
    preddata.to_csv('Data_acsPREDICT7.17_3year.csv',index=False)
    
    return preddata
    


In [59]:
assemble_train_test_predict(years_out=3,year_gap_in_data=2)


[14, 15, 16]
[12, 13, 14, 15]
14
census file path acs_data/acs2010.csv
zillow date 2014-09
current rent date 2010-09
15
16
census file path acs_data/acs2012.csv
zillow date 2016-09
current rent date 2012-09
['do_date', '2012-09']
Index(['RegionName', 'City', 'State', 'Rent', 'nonfamily_households',
       'family_households', 'median_year_structure_built',
       'rent_burden_not_computed', 'rent_over_50_percent',
       'rent_40_to_50_percent',
       ...
       'pop_5_years_over', 'speak_only_english_at_home',
       'speak_spanish_at_home', 'speak_spanish_at_home_low_english',
       'pop_15_and_over', 'pop_never_married', 'pop_now_married',
       'pop_separated', 'pop_widowed', 'pop_divorced'],
      dtype='object', length=254)
13
noteleven
date 2017-09
['do_date', '2013-09']
Index(['RegionName', 'City', 'State', 'Rent', 'nonfamily_households',
       'family_households', 'median_year_structure_built',
       'rent_burden_not_computed', 'rent_over_50_percent',
       'rent_40_to_5

,RegionName,City,State,Rent,nonfamily_households,family_households,median_year_structure_built,rent_burden_not_computed,rent_over_50_percent,rent_40_to_50_percent,...,pop_25_years_over,commute_35_44_mins,commute_60_more_mins,commute_less_10_mins,commuters_16_over,hispanic_any_race,pop_5_years_over,speak_only_english_at_home,speak_spanish_at_home,speak_spanish_at_home_low_english
0,10025-2019-09,New York,NY,3698.0,24367.0,19089.0,1939.0,1754.0,7527.0,2472.0,...,71927.0,6084.0,4294.0,2468.0,45657.0,21349.0,91087.0,59561.0,16641.0,6676.0
1,60657-2019-09,Chicago,IL,1806.0,25743.0,11011.0,1954.0,837.0,4002.0,1476.0,...,52822.0,6809.0,5175.0,1728.0,45212.0,4193.0,65248.0,55901.0,2905.0,785.0
2,10023-2019-09,New York,NY,3521.0,20407.0,12348.0,1944.0,919.0,3221.0,1943.0,...,48957.0,2468.0,1858.0,896.0,30744.0,5880.0,56493.0,41807.0,4269.0,926.0
3,77494-2019-09,Katy,TX,1252.0,3806.0,23386.0,2006.0,57.0,672.0,84.0,...,52360.0,4029.0,6625.0,2256.0,36706.0,15264.0,80564.0,52511.0,11597.0,2367.0
4,60614-2019-09,Chicago,IL,1952.0,22453.0,11252.0,1962.0,696.0,3276.0,1268.0,...,48786.0,4162.0,4247.0,1839.0,42223.0,4634.0,65400.0,56832.0,2580.0,783.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5011,2142-2017-09,Cambridge,MA,3284.0,948.0,437.0,1994.0,61.0,390.0,47.0,...,1981.0,70.0,35.0,184.0,1241.0,293.0,2620.0,1292.0,294.0,8.0
5012,18232-2017-09,Lansford,PA,798.0,683.0,1011.0,1939.0,37.0,205.0,42.0,...,2853.0,168.0,108.0,227.0,1215.0,291.0,3597.0,3337.0,141.0,0.0
5013,75202-2017-09,Dallas,TX,1493.0,1143.0,206.0,1968.0,0.0,91.0,24.0,...,1789.0,137.0,113.0,272.0,1794.0,267.0,2107.0,1572.0,234.0,21.0
5014,60301-2017-09,Oak Park,IL,1513.0,924.0,461.0,1960.0,13.0,190.0,78.0,...,1818.0,232.0,79.0,127.0,1156.0,181.0,2074.0,1688.0,90.0,33.0


['RegionName',
 'City',
 'State',
 'Rent',
 'nonfamily_households',
 'family_households',
 'median_year_structure_built',
 'rent_burden_not_computed',
 'rent_over_50_percent',
 'rent_40_to_50_percent',
 'rent_35_to_40_percent',
 'rent_30_to_35_percent',
 'rent_25_to_30_percent',
 'rent_20_to_25_percent',
 'rent_15_to_20_percent',
 'rent_10_to_15_percent',
 'rent_under_10_percent',
 'total_pop',
 'male_pop',
 'female_pop',
 'median_age',
 'white_pop',
 'black_pop',
 'asian_pop',
 'hispanic_pop',
 'amerindian_pop',
 'other_race_pop',
 'two_or_more_races_pop',
 'not_hispanic_pop',
 'commuters_by_public_transportation',
 'households',
 'median_income',
 'income_per_capita',
 'housing_units',
 'vacant_housing_units',
 'vacant_housing_units_for_rent',
 'vacant_housing_units_for_sale',
 'median_rent',
 'percent_income_spent_on_rent',
 'owner_occupied_housing_units',
 'million_dollar_housing_units',
 'mortgaged_housing_units',
 'families_with_young_children',
 'two_parent_families_with_young_c

## Column Dropping 


In [36]:
traindata=pd.read_csv('Data_acsTRAIN7.17_3year.csv',index_col=False)
testdata=pd.read_csv('Data_acsTEST7.17_3year.csv',index_col=False)
predictdata=pd.read_csv('Data_acsPREDICT7.17_3year.csv',index_col=False)

In [37]:
#list(traindata.columns)


In [38]:
cols_to_drop=[
    
 'nonfamily_households',
 'family_households',
 'median_year_structure_built',
 'rent_burden_not_computed',
 'rent_over_50_percent',
 'rent_40_to_50_percent',
 'rent_35_to_40_percent',
 'rent_30_to_35_percent',
 'rent_25_to_30_percent',
 'rent_20_to_25_percent',
 'rent_15_to_20_percent',
 'rent_10_to_15_percent',
 'rent_under_10_percent',

 'male_pop',
 'female_pop',
 'white_pop',
 'amerindian_pop',
 'other_race_pop',
 'two_or_more_races_pop',
 'not_hispanic_pop',

 'households',
 'vacant_housing_units',
 'median_rent',
 'percent_income_spent_on_rent',
 'owner_occupied_housing_units',

 'families_with_young_children',
 'two_parent_families_with_young_children',
 'two_parents_in_labor_force_families_with_young_children',
 'two_parents_father_in_labor_force_families_with_young_children',
 'two_parents_mother_in_labor_force_families_with_young_children',
 'two_parents_not_in_labor_force_families_with_young_children',
 'one_parent_families_with_young_children',
 'father_one_parent_families_with_young_children',
 'father_in_labor_force_one_parent_families_with_young_children',
 'commute_10_14_mins',
 'commute_15_19_mins',
 'commute_20_24_mins',
 'commute_25_29_mins',
 'commute_30_34_mins',
 'commute_45_59_mins',
 'aggregate_travel_time_to_work', 
 'renter_occupied_housing_units_paying_cash_median_gross_rent',

 'occupied_housing_units',

 'dwellings_1_units_detached',
 'dwellings_1_units_attached',
 'dwellings_2_units',
 'dwellings_3_to_4_units',
 'dwellings_5_to_9_units',
 'dwellings_10_to_19_units',
 'dwellings_20_to_49_units',
 

 'housing_built_2000_to_2004', 
 
 'male_under_5',
 'female_under_5',
 'employed_pop',
 'unemployed_pop', 
 'not_in_labor_force',
 'pop_16_over',
 'pop_in_labor_force',
 'asian_male_45_54',
 'asian_male_55_64',
 'black_male_45_54',
 'black_male_55_64',
 'hispanic_male_45_54',
 'hispanic_male_55_64',
 'white_male_45_54',
 'white_male_55_64', 
 'bachelors_degree_2',
 'bachelors_degree_or_higher_25_64',
 'children',
 'commuters_by_bus',
 'commuters_by_car_truck_van',
 'commuters_by_carpool',
 'commuters_by_subway_or_elevated',
 'commuters_drove_alone',
 'four_more_cars',
 'group_quarters',
 'households_public_asst_or_food_stamps',
 'in_grades_1_to_4',
 'in_grades_5_to_8',
 'in_grades_9_to_12',
 'less_than_high_school_graduate',
 'male_45_64_associates_degree',
 'male_45_64_bachelors_degree',
 'male_45_64_graduate_degree',
 'male_45_64_less_than_9_grade',
 'male_45_64_grade_9_12',
 'male_45_64_high_school',
 'male_45_64_some_college',
 'male_45_to_64', 
 'management_business_sci_arts_employed',
 'graduate_professional_degree', 
 'in_school',
 'no_car',
 'no_cars',  
 'one_car',
 'two_cars',
 'three_cars',
 
 'population_1_year_and_over',
 'population_3_years_over', 
 'workers_16_and_over',
 'high_school_diploma',
 'less_one_year_college',
 'one_year_more_college',
    
    
 'pop_25_years_over',
 'commute_35_44_mins',
 'commuters_16_over',
    
    
 'pop_determined_poverty_status',
 'hispanic_any_race',
    'occupation_natural_resources_construction_maintenance',
 'occupation_production_transportation_material',
 'occupation_services',
'male_male_households',
'female_female_households',
      
    ##now dropping in response to vif/multicolineariy :
    #can try w or without dropping these 
    
    
'civilian_labor_force',
    'sales_office_employed',
    
  

'employed_agriculture_forestry_fishing_hunting_mining',
 'employed_arts_entertainment_recreation_accommodation_food',
 'employed_construction',
 'employed_education_health_social',
 'employed_finance_insurance_real_estate',
 'employed_information',
 'employed_manufacturing',
 'employed_other_services_not_public_admin',
 'employed_public_administration',
 'employed_retail_trade',
 'employed_science_management_admin_waste',
 'employed_transportation_warehousing_utilities',
 'employed_wholesale_trade',
'male_under_5',
 'male_5_to_9',
 'male_10_to_14',
 'male_15_to_17',
 'male_18_to_19',
 'male_20',
 'male_21',
 'male_22_to_24',
 'male_25_to_29',
 'male_30_to_34',
 'male_35_to_39',
 'male_40_to_44',
 'male_45_to_49',
 'male_50_to_54',
 'male_55_to_59',
 'male_65_to_66',
 'male_67_to_69',
 'male_70_to_74',
 'male_75_to_79',
 'male_80_to_84',
 'male_85_and_over',
 'female_under_5',
 'female_5_to_9',
 'female_10_to_14',
 'female_15_to_17',
 'female_18_to_19',
 'female_20',
 'female_21',
 'female_22_to_24',
 'female_25_to_29',
 'female_30_to_34',
 'female_35_to_39',
 'female_40_to_44',
 'female_45_to_49',
 'female_50_to_54',
 'female_55_to_59',
 'female_60_to_61',
 'female_62_to_64',
 'female_65_to_66',
 'female_67_to_69',
 'female_70_to_74',
 'female_75_to_79',
 'female_80_to_84',
 'female_85_and_over',
'housing_units_renter_occupied','mortgaged_housing_units',
    'owner_occupied_housing_units_median_value',
'housing_units','occupation_management_arts','married_households',
    'some_college_and_associates_degree',
   
    'occupation_sales_office',
    'income_less_10000',
 'income_10000_14999',
 'income_15000_19999',
 'income_20000_24999',
 'income_25000_29999',
 'income_30000_34999',
 'income_35000_39999',
 'income_40000_44999',
 'income_45000_49999',
 'income_50000_59999',
 'income_60000_74999',
 'income_75000_99999',
 'income_100000_124999',
 'income_125000_149999',
 'income_150000_199999',
 'income_200000_or_more',
   
    'in_undergrad_college',
    'associates_degree',
    'owner_occupied_housing_units_lower_value_quartile',
    'owner_occupied_housing_units_upper_value_quartile',
    'children_in_single_female_hh','not_us_citizen_pop',
    'black_pop', 'asian_pop', 'hispanic_pop',
       'commuters_by_public_transportation',
'vacant_housing_units_for_rent',
       'vacant_housing_units_for_sale',
    'bachelors_degree', 'masters_degree',
       'commute_60_more_mins', 'commute_less_10_mins','worked_at_home','income_per_capita',
    'pop_25_64',
    'high_school_including_ged','different_house_year_ago_same_city','different_house_year_ago_different_city','housing_built_2005_or_later'

    
]

In [39]:

def combo_and_drop_acs(Tempdata,cols_to_drop):
    Tempdata['Percent_Female']=Tempdata.female_pop/Tempdata.total_pop
    Tempdata['families_with_young_children_summed']=Tempdata['two_parent_families_with_young_children']+Tempdata['two_parents_in_labor_force_families_with_young_children']+Tempdata['two_parents_father_in_labor_force_families_with_young_children']+Tempdata['two_parents_mother_in_labor_force_families_with_young_children']+Tempdata['two_parents_not_in_labor_force_families_with_young_children']+Tempdata['one_parent_families_with_young_children']+Tempdata['father_one_parent_families_with_young_children']+Tempdata['father_in_labor_force_one_parent_families_with_young_children']+Tempdata['families_with_young_children']
    Tempdata['dwellings_1_units']=Tempdata['dwellings_1_units_detached']+Tempdata['dwellings_1_units_attached']
    Tempdata['dwellings_under_10_units']=Tempdata['dwellings_2_units']+Tempdata['dwellings_5_to_9_units']+Tempdata['dwellings_3_to_4_units']
    Tempdata['unemployment_rate']=Tempdata['unemployed_pop']/Tempdata['civilian_labor_force']
    Tempdata['average_car_per_household']=(Tempdata['four_more_cars']*4+Tempdata['three_cars']*3+Tempdata['two_cars']*2+Tempdata['one_car'])/Tempdata.households
    Tempdata['Perc_commuters_under_15_min']=(Tempdata.commute_10_14_mins+Tempdata.commute_less_10_mins)/Tempdata.commuters_16_over
    Tempdata['Perc_commuters_over_45_min']=(Tempdata.commute_60_more_mins+Tempdata.commute_45_59_mins)/Tempdata.commuters_16_over
    Tempdata['Percent_pop_25_64']=Tempdata.pop_25_64/Tempdata.total_pop
    Tempdata['Associates_College_perc']=(Tempdata.associates_degree +Tempdata.bachelors_degree)/Tempdata.total_pop
    Tempdata['Advanced_degree_perc']=(Tempdata.graduate_professional_degree+ Tempdata.masters_degree)/Tempdata.total_pop
    Tempdata['HS_perc']=Tempdata.high_school_including_ged/Tempdata.total_pop
    Tempdata['Perc_vacat_housing_units']=Tempdata.vacant_housing_units /Tempdata.housing_units
    Tempdata['Perc_other_race']=(Tempdata.amerindian_pop+Tempdata.other_race_pop)/Tempdata.total_pop
    Tempdata['Perc_black']=Tempdata.black_pop/Tempdata.total_pop
    Tempdata['Perc_asian']=Tempdata.asian_pop/Tempdata.total_pop
    Tempdata['Perc_hispanic']=Tempdata.hispanic_pop/Tempdata.total_pop
    Tempdata['Perc_commuter_by_public_transport']=Tempdata.commuters_by_public_transportation/Tempdata.commuters_16_over
    Tempdata['Perc_worked_at_home']=Tempdata.worked_at_home/Tempdata.civilian_labor_force
    Tempdata['Perc_different_house_year_ago_same_city']=Tempdata.different_house_year_ago_same_city/Tempdata.total_pop
    Tempdata['Perc_different_house_year_ago_dif_city']=Tempdata.different_house_year_ago_different_city/Tempdata.total_pop
    Tempdata['perc_housing_built_2005_or_later']=Tempdata.housing_built_2005_or_later/Tempdata.housing_units
    return Tempdata.drop(cols_to_drop,axis=1)


In [40]:
testdata=combo_and_drop_acs(testdata,cols_to_drop)
traindata=combo_and_drop_acs(traindata,cols_to_drop)
predictdata=combo_and_drop_acs(predictdata,cols_to_drop)


In [41]:
traindata.columns

Index(['RegionName', 'City', 'State', 'Rent', 'total_pop', 'median_age',
       'median_income', 'million_dollar_housing_units',
       'dwellings_50_or_more_units', 'mobile_homes',
       'housing_built_1939_or_earlier', 'male_60_61', 'male_62_64',
       'white_including_hispanic', 'black_including_hispanic',
       'amerindian_including_hispanic', 'asian_including_hispanic',
       'commute_5_9_mins', 'commute_35_39_mins', 'commute_40_44_mins',
       'commute_60_89_mins', 'commute_90_more_mins',
       'households_retirement_income', 'armed_forces', 'gini_index', 'poverty',
       'walked_to_work', 'pop_5_years_over', 'speak_only_english_at_home',
       'speak_spanish_at_home', 'speak_spanish_at_home_low_english',
       'Percent_Female', 'families_with_young_children_summed',
       'dwellings_1_units', 'dwellings_under_10_units', 'unemployment_rate',
       'average_car_per_household', 'Perc_commuters_under_15_min',
       'Perc_commuters_over_45_min', 'Percent_pop_25_64',
     

In [23]:
#should we percentize race cols
#population density col for zipcode


In [42]:
traindata.to_csv('col_droptrain7.17_3year.csv',index=False)
testdata.to_csv('col_droptest7.17_3year.csv',index=False)
predictdata.to_csv('col_droppredict7.17_3year.csv',index=False)

In [25]:
# Return multicollinearity chart
X = traindata.drop(['Rent', 'RegionName', 'City', 'State'], axis=1)

#used_list=traindata.columns
feature_names = X.columns
RSquared = []
variable_name = []
for var in feature_names:
    dep_var = StandardScaler().fit_transform(np.array(X[var]).reshape(-1,1))
    predictors =StandardScaler().fit_transform(X.drop(var, axis=1))
    ols = LinearRegression()
    ols.fit(predictors, dep_var)
    variable_name.append(var)
    RSquared.append(ols.score(predictors, dep_var))

val_dfs = pd.concat([pd.Series(RSquared), pd.Series(variable_name)], axis=1)
val_dfs = val_dfs.sort_values(by=0, ascending=False)
 
# ## Plotting multicollinearity test from previous project ##
# plt.plot(val_dfs[1], val_dfs[0])
# plt.xticks(rotation=90)
# plt.ylim(0.5,1)
# plt.rcParams["figure.figsize"] = (30,3)
# plt.show()

In [ ]:
val_dfs.head(60)  

In [ ]:
# ##drop notes
#  'nonfamily_households',
#  'family_households',
#  'median_year_structure_built',
#  'rent_burden_not_computed',-hella sparse,want to predict w no rent related cols also for below
#  'rent_over_50_percent', -hella sparse
#  'rent_40_to_50_percent',-hella sparse
#  'rent_35_to_40_percent',-hella sparse
#  'rent_30_to_35_percent',-hella sparse
#  'rent_25_to_30_percent',-hella sparse
#  'rent_20_to_25_percent',-hella sparse
#  'rent_15_to_20_percent',-hella sparse
#  'rent_10_to_15_percent',-hella sparse
#  'rent_under_10_percent',-hella sparse

#  'male_pop', -into percent female
#  'female_pop',-into percent female
#  'white_pop',-should add up to total pop
#  'amerindian_pop',- small 
#  'other_race_pop',-small and redundant, cant drop to offset sum(race) =total pop bc zero in some cases
#  'two_or_more_races_pop',
#  'not_hispanic_pop',-redundant w race cols
#  'commuters_by_public_transportation',
#  'households',-drop bc have family and non family households
#  'vacant_housing_units',-drop have total vacant units for sale and rent
#  'median_rent',-drop 
#  'percent_income_spent_on_rent',- drop cause we predicting this 
#  'owner_occupied_housing_units',- have for sale and rent ,

#  ['families_with_young_children',
#  'two_parent_families_with_young_children',
#  'two_parents_in_labor_force_families_with_young_children',
#  'two_parents_father_in_labor_force_families_with_young_children',
#  'two_parents_mother_in_labor_force_families_with_young_children',
#  'two_parents_not_in_labor_force_families_with_young_children',
#  'one_parent_families_with_young_children',
#  'father_one_parent_families_with_young_children',
#  'father_in_labor_force_one_parent_families_with_young_children',] can all become a fam w children col. the og one doesnt = the sum of all these 
#  'commute_10_14_mins',-drop for commute less than 10 or more than 60
#  'commute_15_19_mins',-drop for commute less than 10 or more than 60
#  'commute_20_24_mins',-drop for commute less than 10 or more than 60
#  'commute_25_29_mins',-drop for commute less than 10 or more than 60
#  'commute_30_34_mins',-drop for commute less than 10 or more than 60
#  'commute_45_59_mins',-drop for commute less than 10 or more than 60
#  'aggregate_travel_time_to_work',-highly population size dependant 
#  'renter_occupied_housing_units_paying_cash_median_gross_rent',-nope

#  'occupied_housing_units',-have for sale and for rent and totoal 

#  ['dwellings_1_units_detached',
#  'dwellings_1_units_attached',
#  'dwellings_2_units',
#  'dwellings_3_to_4_units',
#  'dwellings_5_to_9_units',
#  'dwellings_10_to_19_units',
#  'dwellings_20_to_49_units',
#  'dwellings_50_or_more_units',] can become three cats, one units mid units >50 units 

#  'housing_built_2000_to_2004',-drop- not new anymore so can blend w the other cols 
 
#  'male_under_5',-should sum to total male pop and also maybe issues w child homes so we droppin 
#  'female_under_5',^ same 
#  ['employed_pop',
#  'unemployed_pop',] become unemployment percentage 
#  'not_in_labor_force',- vif w age groups? drop 
#  'pop_16_over',-def vif w age groups
#  'pop_in_labor_force',-interaction w labor force
#  ['asian_male_45_54',
#  'asian_male_55_64',
#  'black_male_45_54',
#  'black_male_55_64',
#  'hispanic_male_45_54',
#  'hispanic_male_55_64',
#  'white_male_45_54',
#  'white_male_55_64',] interact w race and age ranges dropping 
#  'bachelors_degree_2',
#  'bachelors_degree_or_higher_25_64',
#  'children',-vif w age ranges 
#  ['commuters_by_bus',
#  'commuters_by_car_truck_van',
#  'commuters_by_carpool',
#  'commuters_by_subway_or_elevated',
#  'commuters_drove_alone',] - make into drive or public transport 
#  'four_more_cars',
#  'group_quarters',- what this 
#  'households_public_asst_or_food_stamps',
#  ['in_grades_1_to_4',
#  'in_grades_5_to_8',
#  'in_grades_9_to_12',] -will interact w age range
#  'less_than_high_school_graduate',- this with hs degree, bachelors, masters etc should be total pop so dropping this 
#  ['male_45_64_associates_degree',
#  'male_45_64_bachelors_degree',
#  'male_45_64_graduate_degree',
#  'male_45_64_less_than_9_grade',
#  'male_45_64_grade_9_12',
#  'male_45_64_high_school',
#  'male_45_64_some_college',
#  'male_45_to_64',] interact w age and school cols 
#  ['no_car',
#  'no_cars',] combine into avrg cars per person  
#  ['one_car',
#  'two_cars',
#  'three_cars',]combine into avrg cars per person 
#  'pop_25_64',-huge interactions 
#  'population_1_year_and_over',,-huge interactions 
#  'population_3_years_over',,-huge interactions 
#  'workers_16_and_over',-interactions
#  'high_school_diploma',-interactions w hs degree or ged 
#  'less_one_year_college',-interactions
#  'one_year_more_college',-interactions
#  'pop_25_years_over',-interactions
#  'commute_35_44_mins',-drop for commute less than 10 or more than 60
#  'management_business_sci_arts_employed-drop for vif 
#  'graduate_professional_degree-drop bc interacts w other school 
#  'commuters_16_over',-interactions
#  'hispanic_any_race'-interactions]